In [ ]:
import numpy as np
import pandas as pd
import scipy

from IPython.display import clear_output

import sys

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append('../general/src/')

from sklearn.metrics import auc
from scipy.stats import fisher_exact
pd.options.display.precision = 3
pd.options.display.expand_frame_repr = False
pd.options.display.max_columns = 20

In [ ]:
import cv2
import pytesseract

import re
from pathlib import Path
import requests
import random
import time
import scipy
from bs4 import BeautifulSoup

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [ ]:
UserAgent = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1"
        ]


def requestHeader(url):
    # Build request headers
    headers = {
            'User-Agent':random.choice(UserAgent),
            'Referer': url,
            'Connection':'keep-alive'
            }
    return headers

Elsevier

In [ ]:
with open('../secret/elsevier_api.txt') as f:
    api_key = f.read()

In [ ]:
# sciencedirect journals

for journal_name in ['msea']:

    text_df = pd.read_csv('../data/tesseract_ocr_sem_batch_231217_' + journal_name + '.csv')

    meta_df = pd.read_csv('../data/sem_images_231217_batch.csv')

    text_df = pd.merge(meta_df, text_df).copy()

    text_df = text_df.dropna(subset=['text']).copy()

    text_df['instrument_make'] = np.nan

    scope_make_dict = {'TESCAN':'TESCAN',
                        'TESCA':'TESCAN',
                        'TESC':'TESCAN',
                        'VEGA':'TESCAN',
                        'VEGA3':'TESCAN',
                        'VEGAN':'TESCAN',
                        'VEGAS':'TESCAN',
                        'JEOL':'JEOL',
                        'ZEISS':'Zeiss',
                        'KYKY-EM3200':'KYKY',
                        'SU8010':'Hitachi',
                        'SU8000':'Hitachi',
                        'SU3500':'Hitachi',
                        'SU1510':'Hitachi',
                        'SU8020':'Hitachi',
                        'SU5000':'Hitachi',
                        '\$U8000':'Hitachi',
                       '\$U8020':'Hitachi',
                       '\$U8220':'Hitachi',
                        'SU70':'Hitachi',
                        'SU8220':'Hitachi',
                        'S-3400N':'Hitachi',
                        'S-4800':'Hitachi',
                       'S3400N':'Hitachi',
                        'Regulus':'Hitachi',
                        'BSED':'FEI/Thermo',
                        'Quanta':'FEI/Thermo',
                       'QUANTA':'FEI/Thermo',
                        'Nova':'FEI/Thermo',
                        'NanoSEM':'FEI/Thermo',
                       'InBeam':'TESCAN',
                       'AIS2300C':'Seron',
                       '2300C':'Seron',
                       'NOVA':'FEI/Thermo',
                       'GEMINI':'Zeiss',
                       'Apreo':'FEI/Thermo'
                       }

    scope_model_dict = {'VEGA':'TESCAN VEGA',
                       'MIRA':'TESCAN MIRA','GEMINI':'Zeiss Gemini'}

    facility_dict = {'RMRC':'RMRC',
                     'IITM':'IIT Mandi',
                     'Mandi':'IIT Mandi',
                    'IROST':'IROST',
                    'IITB-ChE':'IIT Bombay',
                    'SAIFIITB':'IIT Bombay',
                    'pilani':'BITS Pilani',
                     'Rural':'Gandhigram Rural Insitute',
                     'Gandhigram':'Gandhigram Rural Insitute',
                     'RAZI':'Razi Foundation',
                     'Space':'Insitute of Space Technology',
                     'USIF,':'USIF: Aligarh Muslim University',
                     'SASTRA':'SASTRA',
                     'NRC':'National Research Center', #how to cover 'National Research Center'?
                     '(MUMS)':'Mashhad University',
                     'SUT':'Suranaree University of Technology',
                     'Patiala':'Thapar University',
                     'METU':'Middle East Technical University',
                     'ALIGARH':'USIF: Aligarh Muslim University',
                     'ANNA':'Anna University',
                     'UFSC':'LCME-UFSC',
                     'CRNN':'Calcutta CRNN',
                     '(CIFC)':'IIT BHU',
                     'Amirkabir':'Amirkabir University'
                    }

    for key, value in scope_make_dict.items():
        text_df.loc[text_df['text'].str.contains(key), 'instrument_make'] = value

    instrument_use_by_doi = text_df[~text_df['instrument_make'].isna()]\
    .drop_duplicates(subset=['doi','instrument_make']).groupby('doi')\
    ['instrument_make'].apply(list).apply(lambda x: ', '.join(x))

    n_instrument_use_by_doi = text_df[~text_df['instrument_make'].isna()]\
    .drop_duplicates(subset=['doi','instrument_make']).groupby('doi')\
    ['instrument_make'].apply(list).apply(lambda x: len(x))

    text_by_doi = text_df[~text_df['instrument_make'].isna()]\
    .drop_duplicates(subset=['doi','text']).groupby('doi')\
    ['text'].apply(list).apply(lambda x: ', '.join(x))    

    eval_df = pd.DataFrame(text_by_doi)
    eval_df['instrument_make'] = instrument_use_by_doi
    eval_df['n_instruments'] = n_instrument_use_by_doi

    eval_df = eval_df.reset_index()

    count = 0
    full_text_array = []

    for doi in eval_df['doi'].unique():
        try:
            url = 'https://api.elsevier.com/content/article/doi/' + doi + '?httpAccept=text/plain' + '&APIKey=' + api_key

            response = requests.get(url, headers= requestHeader(url))

            full_text_array.append(pd.DataFrame({'doi':[doi], 'full_text':[response.text]}))
        except:
            print('Error retrieving fulltext for DOI:' + doi)

        count += 1
        print(str(count) + ' of ' + str(eval_df['doi'].nunique()) + ' DOIs processed.')
        if count % 10 == 0:
            clear_output()  

    full_text_df = pd.concat(full_text_array)

    for index, row in full_text_df.iterrows():
        filename = '../data/sem_full_text/' + row['doi'].replace('/','_') + '.txt'

        with open(filename, 'w', encoding='utf') as f:
            f.write(row['full_text'])

    full_text_array = []

    for index, row in eval_df.iterrows():
        doi = row['doi']
        filename = '../data/sem_full_text/' + row['doi'].replace('/','_') + '.txt'

        with open(filename, 'r', encoding='utf') as f:
            full_text = f.read()

        full_text_array.append(pd.DataFrame({'doi':[doi], 'full_text':[full_text]}))

    full_text_df = pd.concat(full_text_array)

    brand_marks = ['TESCAN', 'TE-SCAN', 'TE SCAN', 'tesscan' 
                   'phillips', 'philips', 'jeol', 
                   'ky-ky', 'kyky', 'seron', 
                   'je-ol', 'zeiss', 'nova nano', 'tscan',
                  'fei quanta', 'quanta fei', 'leica', "philip's",
                   'joel', 'cambridge', 'Hithachi', 
                   'hitachi', 'hitashi', '1455VP', '960A', 
                   'XL30', 'XL-30', 'superscan', 'super scan', 
                   'Cam scan', 'camscan', 'MV2300', 'quanta', 
                   'fei', 'nanosem']

    context_str_list = []
    count = 0
    for index, row in full_text_df.iterrows():
        doi = row['doi']
        full_text = row['full_text']
        full_text_lower = full_text.lower()

        context_str_array = []
        match_array = []

        # where references start
        ref_match = re.search('References \[1\]', full_text)
        if not ref_match:
            ref_match = re.search('Reference \[1\]', full_text)
        if not ref_match:
            ref_match = re.search('  References   ', full_text)
        if ref_match:
            ref_start = ref_match.start()
        else:
            ref_match = 10_000_000

        for brand_mark in brand_marks:
            for match in re.finditer(brand_mark.lower(), full_text_lower):
                if True:
                    if (len(match_array) == 0):
                        context_str_array.append(full_text[match.start()-150:match.start()+150])
                        match_array.append(match.start())
                    else:
                        if (np.abs(np.array(match_array) - match.start()) > 150).all():
                            context_str_array.append(full_text[match.start()-150:match.start()+150])
                            match_array.append(match.start())
        context_str = '... ...'.join(context_str_array)
        context_str_list.append(context_str)

        count += 1
        print(str(count) + ' of ' + str(len(eval_df)) + ' DOIs processed.')
        if count % 10 == 0:
            clear_output()

    context_df = pd.DataFrame({'doi': full_text_df['doi'].values, 'context_str':context_str_list})
    context_df = pd.merge(eval_df, context_df)
    context_df['doi'] = context_df['doi'].apply(lambda x: 'https://doi.org/' + x)

    context_df.to_csv('../data/full_text_sem_images_' + journal_name + '_231217.csv', index=False)

PLOS

In [ ]:
# plos journals

for journal_name in ['pone']:

    text_df = pd.read_csv('../data/tesseract_ocr_sem_batch_231204_' + journal_name + '.csv')

    meta_df = pd.read_csv('../data/sem_images_240125_batch.csv')

    text_df = pd.merge(meta_df, text_df).copy()

    text_df = text_df.dropna(subset=['text']).copy()

    text_df['instrument_make'] = np.nan

    scope_make_dict = {'TESCAN':'TESCAN',
                        'TESCA':'TESCAN',
                        'TESC':'TESCAN',
                        'VEGA':'TESCAN',
                        'VEGA3':'TESCAN',
                        'VEGAN':'TESCAN',
                        'VEGAS':'TESCAN',
                        'JEOL':'JEOL',
                        'ZEISS':'Zeiss',
                        'KYKY-EM3200':'KYKY',
                        'SU8010':'Hitachi',
                        'SU8000':'Hitachi',
                        'SU3500':'Hitachi',
                        'SU1510':'Hitachi',
                        'SU8020':'Hitachi',
                        'SU5000':'Hitachi',
                        '\$U8000':'Hitachi',
                       '\$U8020':'Hitachi',
                       '\$U8220':'Hitachi',
                        'SU70':'Hitachi',
                        'SU8220':'Hitachi',
                        'S-3400N':'Hitachi',
                        'S-4800':'Hitachi',
                       'S3400N':'Hitachi',
                        'Regulus':'Hitachi',
                        'BSED':'FEI/Thermo',
                        'Quanta':'FEI/Thermo',
                       'QUANTA':'FEI/Thermo',
                        'Nova':'FEI/Thermo',
                        'NanoSEM':'FEI/Thermo',
                       'InBeam':'TESCAN',
                       'AIS2300C':'Seron',
                       '2300C':'Seron',
                       'NOVA':'FEI/Thermo',
                       'GEMINI':'Zeiss',
                       'Apreo':'FEI/Thermo'
                       }

    scope_model_dict = {'VEGA':'TESCAN VEGA',
                       'MIRA':'TESCAN MIRA','GEMINI':'Zeiss Gemini'}

    facility_dict = {'RMRC':'RMRC',
                     'IITM':'IIT Mandi',
                     'Mandi':'IIT Mandi',
                    'IROST':'IROST',
                    'IITB-ChE':'IIT Bombay',
                    'SAIFIITB':'IIT Bombay',
                    'pilani':'BITS Pilani',
                     'Rural':'Gandhigram Rural Insitute',
                     'Gandhigram':'Gandhigram Rural Insitute',
                     'RAZI':'Razi Foundation',
                     'Space':'Insitute of Space Technology',
                     'USIF,':'USIF: Aligarh Muslim University',
                     'SASTRA':'SASTRA',
                     'NRC':'National Research Center', #how to cover 'National Research Center'?
                     '(MUMS)':'Mashhad University',
                     'SUT':'Suranaree University of Technology',
                     'Patiala':'Thapar University',
                     'METU':'Middle East Technical University',
                     'ALIGARH':'USIF: Aligarh Muslim University',
                     'ANNA':'Anna University',
                     'UFSC':'LCME-UFSC',
                     'CRNN':'Calcutta CRNN',
                     '(CIFC)':'IIT BHU',
                     'Amirkabir':'Amirkabir University'
                    }

    for key, value in scope_make_dict.items():
        text_df.loc[text_df['text'].str.contains(key), 'instrument_make'] = value

    instrument_use_by_doi = text_df[~text_df['instrument_make'].isna()]\
    .drop_duplicates(subset=['doi','instrument_make']).groupby('doi')\
    ['instrument_make'].apply(list).apply(lambda x: ', '.join(x))

    n_instrument_use_by_doi = text_df[~text_df['instrument_make'].isna()]\
    .drop_duplicates(subset=['doi','instrument_make']).groupby('doi')\
    ['instrument_make'].apply(list).apply(lambda x: len(x))

    text_by_doi = text_df[~text_df['instrument_make'].isna()]\
    .drop_duplicates(subset=['doi','text']).groupby('doi')\
    ['text'].apply(list).apply(lambda x: ', '.join(x))    

    eval_df = pd.DataFrame(text_by_doi)
    eval_df['instrument_make'] = instrument_use_by_doi
    eval_df['n_instruments'] = n_instrument_use_by_doi

    eval_df = eval_df.reset_index()

    count = 0
    full_text_array = []

    for doi in eval_df['doi'].unique():
        try:
            url = 'https://doi.org/' + doi

            response = requests.get(url, headers= requestHeader(url))
        
            file = BeautifulSoup(response.text, "lxml")
            
            full_text = ' '.join([p.text for p in file.find_all('p')])
            
            full_text_array.append(pd.DataFrame({'doi':[doi], 'full_text':[full_text]}))
            
            file.decompose()
            
        except:
            print('Error retrieving fulltext for DOI:' + doi)

        count += 1
        print(str(count) + ' of ' + str(eval_df['doi'].nunique()) + ' DOIs processed.')
        if count % 10 == 0:
            clear_output()  

    full_text_df = pd.concat(full_text_array)

    for index, row in full_text_df.iterrows():
        filename = '../data/sem_full_text/' + row['doi'].replace('/','_') + '.txt'

        with open(filename, 'w', encoding='utf') as f:
            f.write(row['full_text'])

    full_text_array = []

    for index, row in eval_df.iterrows():
        doi = row['doi']
        filename = '../data/sem_full_text/' + row['doi'].replace('/','_') + '.txt'

        with open(filename, 'r', encoding='utf') as f:
            full_text = f.read()

        full_text_array.append(pd.DataFrame({'doi':[doi], 'full_text':[full_text]}))

    full_text_df = pd.concat(full_text_array)

    brand_marks = ['TESCAN', 'TE-SCAN', 'TE SCAN', 'tesscan' 
                   'phillips', 'philips', 'jeol', 
                   'ky-ky', 'kyky', 'seron', 
                   'je-ol', 'zeiss', 'nova nano', 'tscan',
                  'fei quanta', 'quanta fei', 'leica', "philip's",
                   'joel', 'cambridge', 'Hithachi', 
                   'hitachi', 'hitashi', '1455VP', '960A', 
                   'XL30', 'XL-30', 'superscan', 'super scan', 
                   'Cam scan', 'camscan', 'MV2300', 'quanta', 
                   'fei', 'nanosem']

    context_str_list = []
    count = 0
    for index, row in full_text_df.iterrows():
        doi = row['doi']
        full_text = row['full_text']
        full_text_lower = full_text.lower()

        context_str_array = []
        match_array = []

        # where references start
        ref_match = re.search('References \[1\]', full_text)
        if not ref_match:
            ref_match = re.search('Reference \[1\]', full_text)
        if not ref_match:
            ref_match = re.search('  References   ', full_text)
        if ref_match:
            ref_start = ref_match.start()
        else:
            ref_match = 10_000_000

        for brand_mark in brand_marks:
            for match in re.finditer(brand_mark.lower(), full_text_lower):
                if True:
                    if (len(match_array) == 0):
                        context_str_array.append(full_text[match.start()-150:match.start()+150])
                        match_array.append(match.start())
                    else:
                        if (np.abs(np.array(match_array) - match.start()) > 150).all():
                            context_str_array.append(full_text[match.start()-150:match.start()+150])
                            match_array.append(match.start())
        context_str = '... ...'.join(context_str_array)
        context_str_list.append(context_str)

        count += 1
        print(str(count) + ' of ' + str(len(eval_df)) + ' DOIs processed.')
        if count % 10 == 0:
            clear_output()

    context_df = pd.DataFrame({'doi': full_text_df['doi'].values, 'context_str':context_str_list})
    context_df = pd.merge(eval_df, context_df)
    context_df['doi'] = context_df['doi'].apply(lambda x: 'https://doi.org/' + x)

    context_df.to_csv('../data/full_text_sem_images_' + journal_name + '_240104.csv', index=False)

Frontiers

In [ ]:
# frontiers journals

for journal_name in ['fchem']:

    text_df = pd.read_csv('../data/tesseract_ocr_sem_batch_240104_' + journal_name + '.csv')

    meta_df = pd.read_csv('../data/sem_images_240104_batch.csv')

    text_df = pd.merge(meta_df, text_df).copy()

    text_df = text_df.dropna(subset=['text']).copy()

    text_df['instrument_make'] = np.nan

    scope_make_dict = {'TESCAN':'TESCAN',
                        'TESCA':'TESCAN',
                        'TESC':'TESCAN',
                        'VEGA':'TESCAN',
                        'VEGA3':'TESCAN',
                        'VEGAN':'TESCAN',
                        'VEGAS':'TESCAN',
                        'JEOL':'JEOL',
                        'ZEISS':'Zeiss',
                        'KYKY-EM3200':'KYKY',
                        'SU8010':'Hitachi',
                        'SU8000':'Hitachi',
                        'SU3500':'Hitachi',
                        'SU1510':'Hitachi',
                        'SU8020':'Hitachi',
                        'SU5000':'Hitachi',
                        '\$U8000':'Hitachi',
                       '\$U8020':'Hitachi',
                       '\$U8220':'Hitachi',
                        'SU70':'Hitachi',
                        'SU8220':'Hitachi',
                        'S-3400N':'Hitachi',
                        'S-4800':'Hitachi',
                       'S3400N':'Hitachi',
                        'Regulus':'Hitachi',
                        'BSED':'FEI/Thermo',
                        'Quanta':'FEI/Thermo',
                       'QUANTA':'FEI/Thermo',
                        'Nova':'FEI/Thermo',
                        'NanoSEM':'FEI/Thermo',
                       'InBeam':'TESCAN',
                       'AIS2300C':'Seron',
                       '2300C':'Seron',
                       'NOVA':'FEI/Thermo',
                       'GEMINI':'Zeiss',
                       'Apreo':'FEI/Thermo'
                       }

    scope_model_dict = {'VEGA':'TESCAN VEGA',
                       'MIRA':'TESCAN MIRA','GEMINI':'Zeiss Gemini'}

    facility_dict = {'RMRC':'RMRC',
                     'IITM':'IIT Mandi',
                     'Mandi':'IIT Mandi',
                    'IROST':'IROST',
                    'IITB-ChE':'IIT Bombay',
                    'SAIFIITB':'IIT Bombay',
                    'pilani':'BITS Pilani',
                     'Rural':'Gandhigram Rural Insitute',
                     'Gandhigram':'Gandhigram Rural Insitute',
                     'RAZI':'Razi Foundation',
                     'Space':'Insitute of Space Technology',
                     'USIF,':'USIF: Aligarh Muslim University',
                     'SASTRA':'SASTRA',
                     'NRC':'National Research Center', #how to cover 'National Research Center'?
                     '(MUMS)':'Mashhad University',
                     'SUT':'Suranaree University of Technology',
                     'Patiala':'Thapar University',
                     'METU':'Middle East Technical University',
                     'ALIGARH':'USIF: Aligarh Muslim University',
                     'ANNA':'Anna University',
                     'UFSC':'LCME-UFSC',
                     'CRNN':'Calcutta CRNN',
                     '(CIFC)':'IIT BHU',
                     'Amirkabir':'Amirkabir University'
                    }

    for key, value in scope_make_dict.items():
        text_df.loc[text_df['text'].str.contains(key), 'instrument_make'] = value

    instrument_use_by_doi = text_df[~text_df['instrument_make'].isna()]\
    .drop_duplicates(subset=['doi','instrument_make']).groupby('doi')\
    ['instrument_make'].apply(list).apply(lambda x: ', '.join(x))

    n_instrument_use_by_doi = text_df[~text_df['instrument_make'].isna()]\
    .drop_duplicates(subset=['doi','instrument_make']).groupby('doi')\
    ['instrument_make'].apply(list).apply(lambda x: len(x))

    text_by_doi = text_df[~text_df['instrument_make'].isna()]\
    .drop_duplicates(subset=['doi','text']).groupby('doi')\
    ['text'].apply(list).apply(lambda x: ', '.join(x))    

    eval_df = pd.DataFrame(text_by_doi)
    eval_df['instrument_make'] = instrument_use_by_doi
    eval_df['n_instruments'] = n_instrument_use_by_doi

    eval_df = eval_df.reset_index()

    count = 0
    full_text_array = []

    for doi in eval_df['doi'].unique():
        try:
            url = 'https://doi.org/' + doi

            response = requests.get(url, headers= requestHeader(url))
        
            file = BeautifulSoup(response.text, "lxml")
            
            full_text = ' '.join([p.text for p in file.find_all('p')])
            
            full_text_array.append(pd.DataFrame({'doi':[doi], 'full_text':[full_text]}))
            
            file.decompose()
            
        except:
            print('Error retrieving fulltext for DOI:' + doi)

        count += 1
        print(str(count) + ' of ' + str(eval_df['doi'].nunique()) + ' DOIs processed.')
        if count % 10 == 0:
            clear_output()  

    full_text_df = pd.concat(full_text_array)

    for index, row in full_text_df.iterrows():
        filename = '../data/sem_full_text/' + row['doi'].replace('/','_') + '.txt'

        with open(filename, 'w', encoding='utf') as f:
            f.write(row['full_text'])

    full_text_array = []

    for index, row in eval_df.iterrows():
        doi = row['doi']
        filename = '../data/sem_full_text/' + row['doi'].replace('/','_') + '.txt'

        with open(filename, 'r', encoding='utf') as f:
            full_text = f.read()

        full_text_array.append(pd.DataFrame({'doi':[doi], 'full_text':[full_text]}))

    full_text_df = pd.concat(full_text_array)

    brand_marks = ['TESCAN', 'TE-SCAN', 'TE SCAN', 'tesscan' 
                   'phillips', 'philips', 'jeol', 
                   'ky-ky', 'kyky', 'seron', 
                   'je-ol', 'zeiss', 'nova nano', 'tscan',
                  'fei quanta', 'quanta fei', 'leica', "philip's",
                   'joel', 'cambridge', 'Hithachi', 
                   'hitachi', 'hitashi', '1455VP', '960A', 
                   'XL30', 'XL-30', 'superscan', 'super scan', 
                   'Cam scan', 'camscan', 'MV2300', 'quanta', 
                   'fei', 'nanosem']

    context_str_list = []
    count = 0
    for index, row in full_text_df.iterrows():
        doi = row['doi']
        full_text = row['full_text']
        full_text_lower = full_text.lower()

        context_str_array = []
        match_array = []

        # where references start
        ref_match = re.search('References \[1\]', full_text)
        if not ref_match:
            ref_match = re.search('Reference \[1\]', full_text)
        if not ref_match:
            ref_match = re.search('  References   ', full_text)
        if ref_match:
            ref_start = ref_match.start()
        else:
            ref_match = 10_000_000

        for brand_mark in brand_marks:
            for match in re.finditer(brand_mark.lower(), full_text_lower):
                if True:
                    if (len(match_array) == 0):
                        context_str_array.append(full_text[match.start()-150:match.start()+150])
                        match_array.append(match.start())
                    else:
                        if (np.abs(np.array(match_array) - match.start()) > 150).all():
                            context_str_array.append(full_text[match.start()-150:match.start()+150])
                            match_array.append(match.start())
        context_str = '... ...'.join(context_str_array)
        context_str_list.append(context_str)

        count += 1
        print(str(count) + ' of ' + str(len(eval_df)) + ' DOIs processed.')
        if count % 10 == 0:
            clear_output()

    context_df = pd.DataFrame({'doi': full_text_df['doi'].values, 'context_str':context_str_list})
    context_df = pd.merge(eval_df, context_df)
    context_df['doi'] = context_df['doi'].apply(lambda x: 'https://doi.org/' + x)

    context_df.to_csv('../data/full_text_sem_images_' + journal_name + '_240104.csv', index=False)

Nature portfolio

In [ ]:
# nature portfolio journals

for journal_name in ['srep']:

    text_df = pd.read_csv('../data/tesseract_ocr_sem_batch_231227_' + journal_name + '.csv')

    meta_df = pd.read_csv('../data/sem_images_231227_batch.csv')

    text_df = pd.merge(meta_df, text_df).copy()

    text_df = text_df.dropna(subset=['text']).copy()

    text_df['instrument_make'] = np.nan

    scope_make_dict = {'TESCAN':'TESCAN',
                        'TESCA':'TESCAN',
                        'TESC':'TESCAN',
                        'VEGA':'TESCAN',
                        'VEGA3':'TESCAN',
                        'VEGAN':'TESCAN',
                        'VEGAS':'TESCAN',
                        'JEOL':'JEOL',
                        'ZEISS':'Zeiss',
                        'KYKY-EM3200':'KYKY',
                        'SU8010':'Hitachi',
                        'SU8000':'Hitachi',
                        'SU3500':'Hitachi',
                        'SU1510':'Hitachi',
                        'SU8020':'Hitachi',
                        'SU5000':'Hitachi',
                        '\$U8000':'Hitachi',
                       '\$U8020':'Hitachi',
                       '\$U8220':'Hitachi',
                        'SU70':'Hitachi',
                        'SU8220':'Hitachi',
                        'S-3400N':'Hitachi',
                        'S-4800':'Hitachi',
                       'S3400N':'Hitachi',
                        'Regulus':'Hitachi',
                        'BSED':'FEI/Thermo',
                        'Quanta':'FEI/Thermo',
                       'QUANTA':'FEI/Thermo',
                        'Nova':'FEI/Thermo',
                        'NanoSEM':'FEI/Thermo',
                       'InBeam':'TESCAN',
                       'AIS2300C':'Seron',
                       '2300C':'Seron',
                       'NOVA':'FEI/Thermo',
                       'GEMINI':'Zeiss',
                       'Apreo':'FEI/Thermo'
                       }

    scope_model_dict = {'VEGA':'TESCAN VEGA',
                       'MIRA':'TESCAN MIRA','GEMINI':'Zeiss Gemini'}

    facility_dict = {'RMRC':'RMRC',
                     'IITM':'IIT Mandi',
                     'Mandi':'IIT Mandi',
                    'IROST':'IROST',
                    'IITB-ChE':'IIT Bombay',
                    'SAIFIITB':'IIT Bombay',
                    'pilani':'BITS Pilani',
                     'Rural':'Gandhigram Rural Insitute',
                     'Gandhigram':'Gandhigram Rural Insitute',
                     'RAZI':'Razi Foundation',
                     'Space':'Insitute of Space Technology',
                     'USIF,':'USIF: Aligarh Muslim University',
                     'SASTRA':'SASTRA',
                     'NRC':'National Research Center', #how to cover 'National Research Center'?
                     '(MUMS)':'Mashhad University',
                     'SUT':'Suranaree University of Technology',
                     'Patiala':'Thapar University',
                     'METU':'Middle East Technical University',
                     'ALIGARH':'USIF: Aligarh Muslim University',
                     'ANNA':'Anna University',
                     'UFSC':'LCME-UFSC',
                     'CRNN':'Calcutta CRNN',
                     '(CIFC)':'IIT BHU',
                     'Amirkabir':'Amirkabir University'
                    }

    for key, value in scope_make_dict.items():
        text_df.loc[text_df['text'].str.contains(key), 'instrument_make'] = value

    instrument_use_by_doi = text_df[~text_df['instrument_make'].isna()]\
    .drop_duplicates(subset=['doi','instrument_make']).groupby('doi')\
    ['instrument_make'].apply(list).apply(lambda x: ', '.join(x))

    n_instrument_use_by_doi = text_df[~text_df['instrument_make'].isna()]\
    .drop_duplicates(subset=['doi','instrument_make']).groupby('doi')\
    ['instrument_make'].apply(list).apply(lambda x: len(x))

    text_by_doi = text_df[~text_df['instrument_make'].isna()]\
    .drop_duplicates(subset=['doi','text']).groupby('doi')\
    ['text'].apply(list).apply(lambda x: ', '.join(x))    

    eval_df = pd.DataFrame(text_by_doi)
    eval_df['instrument_make'] = instrument_use_by_doi
    eval_df['n_instruments'] = n_instrument_use_by_doi

    eval_df = eval_df.reset_index()

    count = 0
    full_text_array = []

    for doi in eval_df['doi'].unique():
        try:
            url = 'https://doi.org/' + doi

            response = requests.get(url, headers= requestHeader(url))
        
            file = BeautifulSoup(response.text, "lxml")
            
            full_text = ' '.join([p.text for p in file.find_all('p')])
            
            full_text_array.append(pd.DataFrame({'doi':[doi], 'full_text':[full_text]}))
            
            file.decompose()
            
        except:
            print('Error retrieving fulltext for DOI:' + doi)

        count += 1
        print(str(count) + ' of ' + str(eval_df['doi'].nunique()) + ' DOIs processed.')
        if count % 10 == 0:
            clear_output()  

    full_text_df = pd.concat(full_text_array)

    for index, row in full_text_df.iterrows():
        filename = '../data/sem_full_text/' + row['doi'].replace('/','_') + '.txt'

        with open(filename, 'w', encoding='utf') as f:
            f.write(row['full_text'])

    full_text_array = []

    for index, row in eval_df.iterrows():
        doi = row['doi']
        filename = '../data/sem_full_text/' + row['doi'].replace('/','_') + '.txt'

        with open(filename, 'r', encoding='utf') as f:
            full_text = f.read()

        full_text_array.append(pd.DataFrame({'doi':[doi], 'full_text':[full_text]}))

    full_text_df = pd.concat(full_text_array)

    brand_marks = ['TESCAN', 'TE-SCAN', 'TE SCAN', 'tesscan' 
                   'phillips', 'philips', 'jeol', 
                   'ky-ky', 'kyky', 'seron', 
                   'je-ol', 'zeiss', 'nova nano', 'tscan',
                  'fei quanta', 'quanta fei', 'leica', "philip's",
                   'joel', 'cambridge', 'Hithachi', 
                   'hitachi', 'hitashi', '1455VP', '960A', 
                   'XL30', 'XL-30', 'superscan', 'super scan', 
                   'Cam scan', 'camscan', 'MV2300', 'quanta', 
                   'fei', 'nanosem']

    context_str_list = []
    count = 0
    for index, row in full_text_df.iterrows():
        doi = row['doi']
        full_text = row['full_text']
        full_text_lower = full_text.lower()

        context_str_array = []
        match_array = []

        # where references start
        ref_match = re.search('References \[1\]', full_text)
        if not ref_match:
            ref_match = re.search('Reference \[1\]', full_text)
        if not ref_match:
            ref_match = re.search('  References   ', full_text)
        if ref_match:
            ref_start = ref_match.start()
        else:
            ref_match = 10_000_000

        for brand_mark in brand_marks:
            for match in re.finditer(brand_mark.lower(), full_text_lower):
                if True:
                    if (len(match_array) == 0):
                        context_str_array.append(full_text[match.start()-150:match.start()+150])
                        match_array.append(match.start())
                    else:
                        if (np.abs(np.array(match_array) - match.start()) > 150).all():
                            context_str_array.append(full_text[match.start()-150:match.start()+150])
                            match_array.append(match.start())
        context_str = '... ...'.join(context_str_array)
        context_str_list.append(context_str)

        count += 1
        print(str(count) + ' of ' + str(len(eval_df)) + ' DOIs processed.')
        if count % 10 == 0:
            clear_output()

    context_df = pd.DataFrame({'doi': full_text_df['doi'].values, 'context_str':context_str_list})
    context_df = pd.merge(eval_df, context_df)
    context_df['doi'] = context_df['doi'].apply(lambda x: 'https://doi.org/' + x)

    context_df.to_csv('../data/full_text_sem_images_' + journal_name + '_231227.csv', index=False)